# Content

This notebook contains a protocol to calculate large amount of normal modes using a highly restricted number of basis (e.g. 1000 vs 28). The 6 free modes were removed with EED(, though there can be more than 6 if there are disconnected components).

In [1]:
import glob
import platform

# A list of pdb available at different sizes

#pdbavail = [ '../../DataRepo/PdbByAtomCount/1a1x.pdb' ] 
pdbavail = sorted(glob.glob( '../../DataRepo/PdbByAtomCount/*.pdb')) + sorted(glob.glob( '../../DataRepo/CifByAtomCount/*.cif'))
Benchmarking_folder = "../../BenchmarkLinuxInchingManyCTRLM0028_RTX4090/"
print(len(pdbavail))

116


# No need to modify anything below, Press Run All

In [2]:

User_Platform = platform.system() # Windows Darwin Linux

User_rc_Gamma = 8.0
User_maxleafsize = 100
User_n_mode = 28
User_tol = 1e-15
User_PlusI = 1.0
PDBCIF = "Pdb" if (".pdb" in pdbavail[0]) else "Cif"
User_MaxIter = 15000

# JDM Params
User_GapEstimate = 1e-6
User_SolverName = 'gmres'
User_SolverMaxIter = 20
User_EigTolerance = 1e-12

User_TotalNumberOfModes = 200
User_TotalNumberOfModes_unchanged = User_TotalNumberOfModes
User_WantedNumberEigenvalue = User_n_mode

User_IntegerOfIndexing = "INTEGER64"

PART00_Import = True
if PART00_Import:
   import os
   import gc
   import sys
   import pickle

   import numpy as np
   import time
   import tqdm

   import torch 


   import platform


   import time

   import cupy
   from cupy import cublas
   import cupy as cp
   import cupyx
   from cupyx.scipy.sparse import linalg as cupylinalg
   from cupyx.scipy import sparse as cupysparse




   sys.path.append('..')
   sys.path.append('../../')
   sys.path.append('../../InchingLiteInteger/Burn/')



   import InchingLiteInteger.util

   InchingLiteInteger.util.MkdirList([Benchmarking_folder])
   
   import InchingLiteInteger.Fuel.Coordinate.T1
   import InchingLiteInteger.Fuel.Coordinate.T2
   import InchingLiteInteger.Burn.Coordinate.T1
   import InchingLiteInteger.Burn.Coordinate.T3

   from InchingLiteInteger.Fuel.T1 import Xnumpy_SparseCupyMatrixUngappped

   import InchingLiteInteger.Burn.Visualisation.T1
   import InchingLiteInteger.Burn.Visualisation.T2


   from InchingLiteInteger.Burn.JacobiDavidsonHotellingDeflation.T1 import S_HeigvalJDMHD_HeigvecJDMHD
   from InchingLiteInteger.Burn.ThickRestartLanczosHotellingDeflation.T1 import S_HeigvalTRLMHD_HeigvecTRLMHD
   from InchingLiteInteger.Burn.ChebyshevDavidsonSubspaceIteration.T1 import S_HeigvalCDSIHD_HeigvecCDSIHD


   import InchingLiteInteger.Burn.HermitianLanczos.T2
   import InchingLiteInteger.Burn.PolynomialFilters.T0
   import InchingLiteInteger.Burn.PolynomialFilters.T2


   # ============================
   # Some torch speed up tips
   # =============================

   # Turn on cuda optimizer
   torch.backends.cudnn.is_available()
   torch.backends.cudnn.enabled = True
   torch.backends.cudnn.benchmark = True
   # disable debugs NOTE use only after debugging
   torch.autograd.set_detect_anomaly(False)
   torch.autograd.profiler.profile(False)
   torch.autograd.profiler.emit_nvtx(False)
   # Disable gradient tracking
   torch.no_grad()
   torch.inference_mode()
   torch.manual_seed(0)
   cupy.random.seed(seed = 0)
   os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # NOTE In case any error showup
   # Reset Cuda and Torch
   device = torch.device(0)
   torch.set_default_dtype(torch.float64)
   torch.set_default_tensor_type(torch.cuda.DoubleTensor)
   try:
      InchingLiteInteger.util.TorchEmptyCache()
   except RuntimeError:
      print("The GPU is free to use. THere is no existing occupant")
   try:
      print(torch.cuda.memory_summary(device = 0, abbreviated=True))
   except KeyError:
      print("The GPU is free to use. THere is no existing occupant")



/home/homingla/anaconda3/envs/V0/envs/Inching23/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The GPU is free to use. THere is no existing occupant
The GPU is free to use. THere is no existing occupant


In [3]:
#pdbavail

In [4]:



for pdbfn in pdbavail[:]:
    
    
    PDBCIF = "Pdb" if (".pdb" in pdbfn) else "Cif"
    devices_ = [d for d in range(torch.cuda.device_count())]
    device_names_  = [torch.cuda.get_device_name(d) for d in devices_]
    User_Device =  device_names_[0]


    pdbid = pdbfn.split("/")[-1].split(".")[0]


    User_TotalNumberOfModes_ = User_TotalNumberOfModes
    CONTINUE_FROM_BEFORE = False
    for ii in range(User_TotalNumberOfModes):#[User_n_mode * (int(User_TotalNumberOfModes/User_n_mode) -1):]:
        if os.path.exists("%s/PerformanceList_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(Benchmarking_folder, pdbid, User_Platform, 
                                                                        User_Device.replace(" ",""), ii)):
            

            with open("%s/Eigval_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(
                        Benchmarking_folder, pdbid, User_Platform,
                        User_Device.replace(" ",""), ii),"rb") as fn:
                eigval = pickle.load(fn)
                current_leftbound_ = np.max(eigval + User_PlusI)-1e-10
                User_IntervalGuess_ = np.abs(eigval[-1] - eigval[0])
            with open("%s/PerformanceList_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(Benchmarking_folder, pdbid, User_Platform, 
                                                                        User_Device.replace(" ",""), ii),"rb") as fn:
                longperformance = pickle.load(fn)
                current_culmruntime_ = longperformance[0][4]


            # NOTE Update count
            User_TotalNumberOfModes_ = User_TotalNumberOfModes - (ii+eigval.shape[0])

            CONTINUE_FROM_BEFORE = True

    #print(CONTINUE_FROM_BEFORE, User_TotalNumberOfModes_, )
    #sys.exit()


    st = time.time()

    X_df, X_top = InchingLiteInteger.util.BasicPdbCifLoading(pdbfn)
    protein_xyz = X_df[['x','y','z']].to_numpy().astype(np.float64)
    # NOTE PDB format digit decimal do no destroy collinearity!
    n_atoms = protein_xyz.shape[0]



    # ===============================================
    # K-d Cuthill (NOTE CPU np array)
    # ===================================
    # NOTE Cuthill Order and Undo
    st = time.time()
    cuthill_order, cuthill_undoorder = InchingLiteInteger.Fuel.Coordinate.T1.X_KdCuthillMckeeOrder(protein_xyz,  
                                rc_Gamma = User_rc_Gamma, Reverse = True,
                                )
    protein_xyz = protein_xyz[cuthill_order,:]




    print('start eigsh cupy')

    mempool = cupy.get_default_memory_pool()
    pinned_mempool = cupy.get_default_pinned_memory_pool()

    # ==================
    # Cupy hessian
    # =====================
    PART03a_MakeCupyHessian = True
    if PART03a_MakeCupyHessian:
        # NOTE Nnz neighborhood after cuthill
        NnzMinMaxDict, HalfNnz  = InchingLiteInteger.Fuel.Coordinate.T1.X_KdUngappedMinMaxNeighbor(protein_xyz, 
                                    rc_Gamma = User_rc_Gamma, 
                                    maxleafsize = User_maxleafsize,
                                    CollectStat = False,
                                    User_ReturnHalfNnz = True,
                                    SliceForm= True)


        # NOTE Pyotch tensor spend textra memory when dlpack has to be called and there are mmeleak
        #X = torch.tensor(protein_xyz, device=device, requires_grad= False)
        X = protein_xyz

        Xnumpy_SparseCupyMatrixUngapppedC = Xnumpy_SparseCupyMatrixUngappped(X, batch_head = None,
            maxleafsize = User_maxleafsize, rc_Gamma = User_rc_Gamma,
            #device  = torch.device(0),
            User_PlusI = User_PlusI,
            #dtype_temp = torch.float64,
            #X_precision = torch.cuda.DoubleTensor,
            User_DictCharmmGuiPbc = None, #Dict_Pbc,
            NnzMinMaxDict = NnzMinMaxDict)
        if User_IntegerOfIndexing == "INTEGER32":
            A, A_diag = Xnumpy_SparseCupyMatrixUngapppedC.ReturnCupyHLowerTriangleInt32(
                            User_MaxHalfNnzBufferSize = HalfNnz)
        else:
            #print('gagag')
            A, A_diag = Xnumpy_SparseCupyMatrixUngapppedC.ReturnCupyHLowerTriangleInt64(
                            User_MaxHalfNnzBufferSize = HalfNnz)

        print("Matrix Index Datatype", A.indices.dtype)
        print("Matrix Datatype",A.data.shape)
        
        cupy.get_default_memory_pool().free_all_blocks()
        cupy.get_default_pinned_memory_pool().free_all_blocks()
        gc.collect()





    PART03b_MakeFreeModes = True
    if PART03b_MakeFreeModes:

        Q_HotellingDeflation = cp.zeros((6,3*n_atoms), dtype = cp.float64)
        # NOTE Translation
        for i in range(3):
            q1 = cp.zeros((n_atoms,3))
            q1[:,i] = 1/np.sqrt(n_atoms)
            Q_HotellingDeflation[i,:] = q1.flatten()
            q1 = None
            del q1
            cupy.get_default_memory_pool().free_all_blocks()
            cupy.get_default_pinned_memory_pool().free_all_blocks()



        # NOTE Rotation
        R_x = cp.array([        [0,0,0],
                                [0,0,-1],
                                [0,1,0]], dtype=cp.float64).T
        R_y = cp.array([        [0,0,1],
                                [0,0,0],
                                [-1,0,0]], dtype=cp.float64).T
        R_z = cp.array([        [0,-1,0],
                                [1,0,0],
                                [0,0,0]], dtype=cp.float64).T
        R_x = cupysparse.csr_matrix(R_x, dtype= cp.float64)
        R_y = cupysparse.csr_matrix(R_y, dtype= cp.float64)
        R_z = cupysparse.csr_matrix(R_z, dtype= cp.float64)
        gx = (cp.array(X)@R_x).flatten()
        Q_HotellingDeflation[3,:] = gx/ cp.linalg.norm(gx,ord=2)
        gy = (cp.array(X)@R_y).flatten()
        Q_HotellingDeflation[4,:] = gy/ cp.linalg.norm(gy,ord=2)
        gz = (cp.array(X)@R_z).flatten()
        Q_HotellingDeflation[5,:] = gz/ cp.linalg.norm(gz,ord=2)



        for i_FRO in range(2):
            V = Q_HotellingDeflation.T

            for ix in range(6):
                if ix == 0:
                    continue
                V[:,ix] -= cp.matmul(V[:,:ix], cp.matmul( V[:, :ix].T,V[:,ix] ))
                V[:,ix] /= cp.sqrt(V[:, ix].T @ V[:, ix]) # TODO torch.matmul or mvs
                V[:,ix] -= cp.matmul(V[:,:ix], cp.matmul( V[:, :ix].T,V[:,ix] ))
                V[:,ix] /= cp.sqrt(V[:, ix].T @ V[:, ix])
            Q_HotellingDeflation = V.T

        #gx = Q_HotellingDeflation[3]

        # NOTE Because of the naiive pbc implemented only 3 translational mode matters.
        Q_HotellingDeflation = cupyx.scipy.sparse.csr_matrix(Q_HotellingDeflation, dtype = cp.float64)

        gx, gy, gz = None, None, None
        del gx, gy, gz
        cupy.get_default_memory_pool().free_all_blocks()
        cupy.get_default_pinned_memory_pool().free_all_blocks()



    # ==============================
    # Main loop
    # ================================

    User_SpectrumBound = InchingLiteInteger.Burn.HermitianLanczos.T2.A_Adiag_EstimateSpectrumBound(
                            A, A_diag, User_HalfMemMode = True )
    RitzValues = User_SpectrumBound[2]
    #print(User_SpectrumBound)
    print(float(np.quantile(RitzValues,0.025)))
    User_SpectrumBound = (User_SpectrumBound[0], User_SpectrumBound[1])
    User_SpectrumBound = (User_PlusI, User_SpectrumBound[1].get()+1e-12)

    remaining_n_modes = User_TotalNumberOfModes_

    print("REMAINING" ,pdbid, remaining_n_modes)
    if not CONTINUE_FROM_BEFORE:
        current_leftbound = User_SpectrumBound[0]
        current_culmruntime = 0
        User_IntervalGuess = 0.25
    else:
        current_leftbound = current_leftbound_
        current_culmruntime = current_culmruntime_
        User_IntervalGuess = User_IntervalGuess_
    SLEEPTIME = 60



    #print(remaining_n_modes, User_TotalNumberOfModes_unchanged, np.max(eigval))
    #print((current_leftbound, current_leftbound+User_IntervalGuess*3.0))
    #sys.exit()
    while remaining_n_modes > 0:
        time.sleep(SLEEPTIME)
        PART04a_PolynomialFilter = True
        if PART04a_PolynomialFilter:

                if remaining_n_modes == User_TotalNumberOfModes_unchanged:
                    """
                    # NOTE We will not use kpm at extremal, but take the heuristic
                    if PDBCIF == "Pdb":
                        User_WantedInterval_ = (1.0, 8.0)
                    else:
                        User_WantedInterval_ = (1.0, 1.1)

                    User_WantedInterval =  User_WantedInterval_
                    User_PolynomialParams = InchingLiteInteger.Burn.PolynomialFilters.T0.OOC0_OptimizePolynomialParamsOnDegree(
                             User_MaximumDegree = 5120,
                             User_MinimumDegree = 5,
                             User_DampingKernel = "Jackson",
                             User_ExtremalIntervalDefinition = 1e-10,
                             User_WantedInterval = User_WantedInterval,
                             User_SpectrumBound = User_SpectrumBound,
                             User_ConvergenceRatio = 0.1,
                             )
                    temp_User_WantedInterval = User_WantedInterval
                    """


                    User_WantedInterval_ = (1.0, User_SpectrumBound[1]/10)
                    User_WantedInterval = User_WantedInterval_
                    # NOTE For the extremal it is empirically unwise to squeeze too hard as many of 
                    #      these eigenvalues are close to 0+1 and the mapped function is in steep decline (with very deep caution here.)
                    #      I would delibrately make the wanted nubmer of eigval larger, because anyhow the lowest will be the first to converge in this scenairo
                    User_PolynomialParams, eigval_count_estimate, temp_User_WantedInterval = InchingLiteInteger.Burn.PolynomialFilters.T2.A_Adiag_OptimizePolynomialParamsOnMemory(
                                                                    A, A_diag,
                                                                    User_MaximumDegree = 5120,
                                                                    User_MinimumDegree = 5,
                                                                    User_DampingKernel = "Jackson",
                                                                    User_ExtremalIntervalDefinition = 1e-10,
                                                                    User_WantedInterval = User_WantedInterval,
                                                                    User_SpectrumBound = User_SpectrumBound,
                                                                    User_DesignatedStart = User_SpectrumBound[1]/10,
                                                                    # NOTE Make this a underestimate of the actual number.
                                                                    #      While we will have repeated modes, the risk of having a repeated mode is outweighed by risk of non-convergence!
                                                                    #
                                                                    User_WantedNumberEigenvalue = int(User_WantedNumberEigenvalue-5)*5,
                                                                    User_AffordableMemoryMargin = 5*5,
                                                                    User_HalfMemMode = True,
                                                                    User_NumberKpmTrials =  5,
                                                                    User_ConvergenceRatio = 0.7, # NOTE For extremal this is overrided with 0.1
                                                                    )
                    User_WantedInterval = (temp_User_WantedInterval[0], temp_User_WantedInterval[1])
                else:
                    #User_WantedInterval = (current_leftbound, current_leftbound+3) # NOTE Example
                    User_WantedInterval = (current_leftbound, current_leftbound+User_IntervalGuess*3.0)
                    print(User_SpectrumBound, User_WantedInterval)

                    # NOTE Find a memory-fit interval
                    User_PolynomialParams, eigval_count_estimate, temp_User_WantedInterval = InchingLiteInteger.Burn.PolynomialFilters.T2.A_Adiag_OptimizePolynomialParamsOnMemory(
                                                                    A, A_diag,
                                                                    User_MaximumDegree = 5120,
                                                                    User_MinimumDegree = 5,
                                                                    User_DampingKernel = "Jackson",
                                                                    User_ExtremalIntervalDefinition = 1e-10,
                                                                    User_WantedInterval = User_WantedInterval,
                                                                    User_SpectrumBound = User_SpectrumBound,
                                                                    User_DesignatedStart = User_WantedInterval[0]+User_IntervalGuess*3.0,
                                                                    # NOTE Make this a underestimate of the actual number.
                                                                    #      While we will have repeated modes, the risk of having a repeated mode is outweighed by risk of non-convergence!
                                                                    #
                                                                    User_WantedNumberEigenvalue = int(User_WantedNumberEigenvalue-5),
                                                                    User_AffordableMemoryMargin = 5,
                                                                    User_HalfMemMode = True,
                                                                    User_NumberKpmTrials =  5,
                                                                    User_ConvergenceRatio = 0.7, # NOTE For extremal this is overrided with 0.1
                                                                    )


                # NOTE Update
                #User_WantedNumberEigenvalue = eigval_count_estimate
                #User_WantedInterval = tuple([User_WantedInterval[0], temp_User_WantedInterval[1]])


                print(User_PolynomialParams.AdjustedDegree, temp_User_WantedInterval)


        PART04_CalcualteEig = True
        if PART04_CalcualteEig:

            if remaining_n_modes == User_TotalNumberOfModes_unchanged:
                # NOTE Let's pull away the (basic) free mode. 
                #      This is useful when the non-rigid modes are really close to 1 and when you cannot make distinction. 
                #      However, it will 'fail' when there are disconnected component as there are more than 6 free mdes.
                User_Q_HotellingDeflation_ = Q_HotellingDeflation
            else:
                User_Q_HotellingDeflation_ = None



            eigval, eigvec = S_HeigvalTRLMHD_HeigvecTRLMHD(A, A_diag,


                        User_WorkspaceSizeFactor = 2 ,
                        k = User_n_mode ,
                        tol = User_EigTolerance,
                        maxiter = User_MaxIter,
                        #User_CorrectionSolverMaxiter = User_SolverMaxIter,
                        User_HalfMemMode= True,
                        #User_IntermediateConvergenceTol=1e-3, # NOTE Do not touch for this problem
                        #User_GapEstimate = User_GapEstimate, # NOTE This will be used for theta - gap_estimate
                        #User_FactoringToleranceOnCorrection = 1e-4,#1e-4, # NOTE Do not touch for this problem

                        User_Q_HotellingDeflation = User_Q_HotellingDeflation_, #Q_HotellingDeflation,
                        User_HotellingShift = -40.0,# NOTE pull to negative, we are on cheb

                        User_PolynomialParams = User_PolynomialParams,
                        )

            User_IntervalGuess = np.abs(cupy.asnumpy(eigval)[-1] - cupy.asnumpy(eigval)[0])

            runtime = time.time() - st
            print("RUNNNTIME %s" %(runtime))
            peak_mem = cupy.get_default_memory_pool().used_bytes() / 1024 / 1024


            runtime = time.time() - st
            peak_mem = cupy.get_default_memory_pool().used_bytes() / 1024 / 1024

            print(eigval)
            print(User_WantedInterval)
            # =============================
            # Save only relevants
            # =============================
            # NOTE We do not discuss cases where there are overlap e.g. due to pseudo symmetry for simplicity.
            idx_relevant = np.where(
                (User_WantedInterval[0] < cupy.asnumpy(eigval)
                 ) )[0]

            eigval = eigval[idx_relevant]
            eigvec = eigvec[:,idx_relevant]
            with open("%s/Eigval_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(
                        Benchmarking_folder, pdbid, User_Platform,
                        User_Device.replace(" ",""), int(User_TotalNumberOfModes - remaining_n_modes)),"wb") as fn:
                pickle.dump(cupy.asnumpy(eigval) - User_PlusI ,fn, protocol=4)
            """
            with open("%s/Eigvec_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(
                        Benchmarking_folder, pdbid, User_Platform,
                        User_Device.replace(" ","") ,int(User_TotalNumberOfModes - remaining_n_modes)),"wb") as fn:
                tempeigvec = cupy.asnumpy(eigvec)
                tempeigvec = tempeigvec.T
                tempeigvec = tempeigvec.reshape((int(User_n_mode),int(n_atoms),int(3)))
                pickle.dump(tempeigvec[:,cuthill_undoorder,:] ,fn, protocol=4)

            del tempeigvec
            """
            gc.collect()





        current_leftbound = np.max(cupy.asnumpy(eigval)) - 1e-10 # NOTE To prevent stepping right on the first eigenvalue. thouhg it also means that we need to remove repeated calcualteinos


        PART05_Performance = True
        if PART05_Performance:
            #===================================
            # Check correct
            # =====================================

            User_HalfMemMode = True
            if User_HalfMemMode:
                KrylovAv = InchingLiteInteger.Burn.Krylov.T3.OOC2_HalfMemS_v_KrylovAv_VOID(A, A_diag)
            else:
                KrylovAv = InchingLiteInteger.Burn.Krylov.T3.OOC2_FullMemS_v_KrylovAv_VOID(A, A_diag)
            Av = cupy.empty((n_atoms*3,)).astype(A.dtype)


            delta_lambda_list = []
            for jj in range(eigval.shape[0]  ):
                KrylovAv(A,cupy.ravel(eigvec[:,jj]),Av)
                B = Av - eigval[jj]* cupy.ravel(eigvec[:,jj])

                delta_lambda_list.append(cupy.asnumpy(cublas.nrm2(B)))
                #if jj < 20:
                print(eigval[jj], cupy.asnumpy(cublas.nrm2(B)))

            eigval = cupy.asnumpy(eigval)
            n_atoms = protein_xyz.shape[0]

            GPU = "%s %s" %(User_Platform, User_Device.replace(" GPU", ""))

            performance = ["Inching (CTRLM Interior %s)" %(GPU), pdbfn, n_atoms,
                            runtime + current_culmruntime - SLEEPTIME, peak_mem,
                            User_Platform, User_Device,
                            User_maxleafsize]

            #sys.exit()

            longperformance = []
            for i in range(len(delta_lambda_list)):
                longperformance.append(performance + [i ,
                                                      delta_lambda_list[i], eigval[i] - User_PlusI, 
                                                      int(User_PolynomialParams.AdjustedDegree), 
                                                      int(User_TotalNumberOfModes - remaining_n_modes)])

            with open("%s/PerformanceList_InchingManyCTRLM_%s_%s_%s_%s.pkl" %(Benchmarking_folder,
                pdbid, User_Platform, User_Device.replace(" ",""), int(User_TotalNumberOfModes - remaining_n_modes)),"wb") as fn:
                pickle.dump(longperformance,fn, protocol=4)

            print("REMAINING %s has %s atoms. Modes remaining %s " %(pdbid, n_atoms, remaining_n_modes- len(delta_lambda_list)))
            #del X_df, protein_xyz
            gc.collect()


            eigvec, eigval = None, None
            del eigvec, eigval


            cupy.get_default_memory_pool().free_all_blocks()
            cupy.get_default_pinned_memory_pool().free_all_blocks()

            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats(0)
            torch.cuda.memory_allocated(0)
            torch.cuda.max_memory_allocated(0)
        remaining_n_modes -= len(delta_lambda_list)
        #remaining_n_modes = max(0, remaining_n_modes)
        
    PART05_CleanseMem = True
    if PART05_CleanseMem:
            del X_df, protein_xyz
            gc.collect()

            B = None
            A.data = None
            A.indices = None
            A.indptr = None
            Q_HotellingDeflation = None
            del Q_HotellingDeflation

            del A.data, A.indices, A.indptr
            del A, B
            Xnumpy_SparseCupyMatrixUngapppedC.X, Xnumpy_SparseCupyMatrixUngapppedC.X_unsqueezed = None, None
            del Xnumpy_SparseCupyMatrixUngapppedC.X, Xnumpy_SparseCupyMatrixUngapppedC.X_unsqueezed
            Xnumpy_SparseCupyMatrixUngapppedC = None
            del Xnumpy_SparseCupyMatrixUngapppedC
            del X
            cupy.get_default_memory_pool().free_all_blocks()
            cupy.get_default_pinned_memory_pool().free_all_blocks()
            torch.cuda.empty_cache()
            torch.cuda.reset_peak_memory_stats(0)
            torch.cuda.memory_allocated(0)
            torch.cuda.max_memory_allocated(0)

            time.sleep(120) # NOTE Sleep longer for the sake of health!


On Linux


100%|██████████| 4/4 [00:00<00:00, 241.05it/s]


N_neighbor within 8.0 angstrom Mean 81.3613487669854, Std 24.04968384820146
NN search in 0.018742084503173828 s


  0%|          | 0/3974 [00:00<?, ?it/s]


start eigsh cupy
On Linux


...

Kpm Estiamte Mean 42.2 Std 8.182909018191515
Adjusted degree to 1249 and Interval to 1.0294533876284313,1.0363813309780268. Estimate number of eigval is 43
Convergence ratio 0.7


100%|██████████| 5/5 [03:20<00:00, 40.19s/it]


Kpm Estiamte Mean 15.6 Std 4.317406628984581
Adjusted degree to 2435 and Interval to 1.0294533876284313,1.032917359303229. Estimate number of eigval is 17
Convergence ratio 0.7


100%|██████████| 5/5 [02:16<00:00, 27.34s/it]


Kpm Estiamte Mean 26.0 Std 6.2289646009589745
Adjusted degree to 1645 and Interval to 1.0294533876284313,1.034649345140628. Estimate number of eigval is 27
Convergence ratio 0.7
1645 (1.0294533876284313, 1.034649345140628)
There are 56 Ritz vectors, tol = 1e-12
WARNING. Hotelling deflation not in use
